In [ ]:
# 全部用selenium抓到台積電基本資料
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
import time,bs4,requests

url='https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5'
driver_path=Service('D:\chromedriver.exe')
driver=webdriver.Chrome(service=driver_path)
driver.get(url)

driver.find_element(By.CSS_SELECTOR,'.vector-search-box-input').send_keys('台積電')
driver.find_element(By.CSS_SELECTOR,'#searchButton').click()
obj_soup=bs4.BeautifulSoup(driver.page_source,'lxml')

obj_title=driver.find_element(By.CSS_SELECTOR,'#content h1')
obj_title=obj_title.text.split('[')[0]
print(obj_title)
tome.sleep(1)

obj_type=driver.find_element(By.CSS_SELECTOR,'#bodyContent #siteSub')
print(obj_type.text)
tome.sleep(1)

obj_content=driver.find_element(By.CSS_SELECTOR,'.mw-parser-output p')
print(obj_content.get_attribute('textContent'))
tome.sleep(1)

driver.close()

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
import time,bs4

url='https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5'
driver_path=Service('D:\chromedriver.exe')
driver=webdriver.Chrome(service=driver_path)
driver.get(url)

driver.find_element(By.CSS_SELECTOR,'.vector-search-box-input').send_keys('台積電')
driver.find_element(By.CSS_SELECTOR,'#searchButton').click()
obj_soup=bs4.BeautifulSoup(driver.page_source,'lxml')

obj_title=obj_soup.find('div','mw-body')
print(obj_title.h1.text.strip())
time.sleep(1)

obj_type=obj_soup.find('div',id='siteSub').text.strip()
print(obj_type)
time.sleep(1)

# 以下三種方式都可以找到第一個p標籤內的字串
obj_content_1=obj_title.find('p')
obj_content_2=obj_soup.find('div','mw-parser-output').find_all('p')[0].text.strip()
obj_content_3=obj_title.find_all('p')
print(obj_content_1.text.strip())
print(obj_content_2)
print(obj_content_3[0].text.strip())
time.sleep(1)

driver.close()

In [ ]:
# 爬文台積電簡史 bs4直接進網址抓 練習用pd轉不同方向df
import bs4,requests
import pandas as pd

#url='https://zh.wikipedia.org/wiki/%E5%8F%B0%E7%81%A3%E7%A9%8D%E9%AB%94%E9%9B%BB%E8%B7%AF%E8%A3%BD%E9%80%A0'
url='https://zh.wikipedia.org/wiki/台灣積體電路製造'  # 使用中文網址也可以
headers={'user-agent': 'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'}
html=requests.get(url,headers=headers)
obj_soup=bs4.BeautifulSoup(html.text,'lxml')

obj_list,name_list,content_list=[],[],[]
obj_dict={}

obj_tsmc=obj_soup.find('table','infobox').tbody.find_all('tr')
for tsmc in obj_tsmc:
    #print(tsmc.text.strip())
    # 如果裡面的標籤不同時 可用此方法印出
    col=tsmc.find('th','fn org')
    if col:
        name=col.text.strip()
        print(name)
    col=tsmc.find('td',style='text-align:center')
    if col:
        print(col.text.strip())
    col=tsmc.find('th',scope='row')
    if col:
        print(col.text.strip(),':',tsmc.td.text.strip())
        
        # 轉list
        obj_list.append([col.text.strip(),tsmc.td.text.strip()])
        
        # 轉字典變成標題在上 方便合併很多公司
        name_list.append(col.text.strip())
        content_list.append(tsmc.td.text.strip())
        obj_dict={name:content_list}
        
# 轉list
list_df=pd.DataFrame(obj_list,columns=['名稱',name])
list_df.set_index('名稱',inplace=True)
#list_df.to_csv('company.csv',index=False)
print(list_df)
print('-'*30)
print(list_df.T)

# 轉字典變成標題在上 方便合併很多公司 但前提是columns要完全一樣
print(obj_dict)
print()
dict_df=pd.DataFrame(obj_dict.values(),index=obj_dict.keys(),columns=name_list)
#dict_df.to_csv('company.csv')
print(dict_df)

In [ ]:
# 爬文台積電及鴻海簡史 selenium輸入抓 用list 合併兩個不同columns的df
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
import time,bs4,requests
import pandas as pd

url='https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5'
driver_path=Service('D:\chromedriver.exe')
driver=webdriver.Chrome(service=driver_path)
driver.get(url)

company_list=['台積電','叡揚資訊']   #,'鴻海','華碩'
big_name_list=[list() for n in range(len(company_list))]
big_content_list=[list() for n in range(len(company_list))]

for c in company_list:
    driver.find_element(By.CSS_SELECTOR,'.vector-search-box-input').send_keys(c)
    driver.find_element(By.CSS_SELECTOR,'#searchButton').click()
    time.sleep(2)
    # 因為每個公司的資訊標題不同 所以要轉成list區隔 
    name_list,content_list=[],[]
    # 每次新增到大list後要清空在放下一個公司的資訊
    name_list.clear()
    content_list.clear()
    
    obj_soup=bs4.BeautifulSoup(driver.page_source,'lxml')
    obj_info=obj_soup.find('table','infobox').tbody.find_all('tr')
    for info in obj_info:
        index=info.find('th','fn org')
        en_index=info.find('td',style='text-align:center')
        th_row=info.find('th',scope='row')
        td=info.td
        # 如果tr內有符合 公司名稱 的標籤 
        if index:    
            print(index.text.strip())
        # 如果tr內有符合 公司英文名稱 的標籤
        if en_index:  
            print(en_index.text.strip())
        # 如果tr內有符合 公司資訊 的標籤
        if th_row and td:
            print(th_row.text.strip(),'-',td.text.strip())
            # 先把公司資訊加到list
            name_list.append(th_row.text.strip())  # 標題
            content_list.append(td.text.strip())   # 內容
    # 再加到大list 以做區隔
    big_name_list.append(name_list)
    big_content_list.append(content_list)

    print('-'*30)
    print(big_name_list)
    print(big_content_list)

driver.close()

In [ ]:
import pandas as pd

big_name_list=[['公司類型', '股票代號', 'ISIN', '統一編號', '成立', '創辦人', '代表人物', '總部', '產業', '產品', '營業額', '稅前盈餘', '稅後盈餘', '總資產', '資產淨值', '員工人數', '實收資本額', '市值', '每股盈利', '每股資產淨值', '主要股東', '主要子公司', '網站'], ['統一編號', '成立', '創辦人', '總部', '標語口號', '產業', '員工人數', '網站']]
big_content_list=[['上市公司', '臺證所：2330（1994年9月5日上市）NYSE：TSM（1997年10月20日上市）LSE：0LCVFWB：TSFA', 'US8740391003', '22099131  （查）', '1987年2月21日\xa0(1987-02-21)', '張忠謀', '董事長：劉德音副董事長兼總裁：魏哲家', '臺灣 300096新竹市東區力行六路8號', '電子工業', '晶圓代工服務', '▲新臺幣1.58兆元（2021年）[1]', '▲新臺幣5,847億元（2020年）', '▲新臺幣5,181億元（2020年）', '▲新臺幣2.76兆元（2020年）', '65,376,200,000 美元 (2020年)', '▲50,000人（2020年）', '新臺幣259,303,804,580元', '▲新臺幣16.38兆元（2021年2月5日）', '▲新臺幣19.97元（2020年）', '▲新臺幣71.33元（2020年）', '美國花旗銀行託管台積電存託憑證專戶（21.8%）行政院國家發展基金管理會（6.38%）管理人員群體（0.36%）[2]', '30家[3]', 'www.tsmc.com'], ['22425662  （查）', '1987年', '張培鏞', '中華民國（臺灣） 台北市中山區德惠街9號5樓', '品質與價值，承諾必實現Quality & value , we are committed', '資訊服務', '約700人', 'gss.com.tw']]
print(big_name_list)
print(big_content_list)

# df=pd.DataFrame([big_content_list[0]],columns=big_name_list[0])

df=[pd.Series() for n in range(len(company_list))]
df1=pd.DataFrame([big_content_list[0]],columns=big_name_list[0])
df2=pd.DataFrame([big_content_list[1]],columns=big_name_list[1])
# print(df1)
# print(df2)

# for r in range(len(big_content_list)):
#     df1=pd.DataFrame([big_content_list[r]],columns=big_name_list[r])
#     print(df[r])

In [3]:
# 爬文台積電及鴻海簡史 selenium輸入抓 用dict 合併相同key變df
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
import time,bs4,requests
import pandas as pd

url='https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5'
driver_path=Service('D:\chromedriver.exe')
driver=webdriver.Chrome(service=driver_path)
driver.get(url)

company_name_list=['台積電','叡揚資訊','華碩']   #,'鴻海','華碩'
big_company_list=[]
big_name_list=[list() for n in range(len(company_name_list))]
big_content_list=[list() for n in range(len(company_name_list))]

for c in company_name_list:
    driver.find_element(By.CSS_SELECTOR,'.vector-search-box-input').send_keys(c)
    driver.find_element(By.CSS_SELECTOR,'#searchButton').click()
    time.sleep(2)
    # 因為每個公司的資訊標題不同 所以要轉成list區隔 
    name_list,content_list=[],[]
    # 每次新增到大list後要清空在放下一個公司的資訊
    name_list.clear()
    content_list.clear()
    
    obj_soup=bs4.BeautifulSoup(driver.page_source,'lxml')
    obj_info=obj_soup.find('table','infobox').tbody.find_all('tr')
    for info in obj_info:
        index=info.find('th','fn org')
        en_index=info.find('td',style='text-align:center')
        th_row=info.find('th',scope='row')
        td=info.td
        # 如果tr內有符合 公司名稱 的標籤 
        if index:    
            print(index.text.strip())
            name=index.text.strip()
        # 如果tr內有符合 公司英文名稱 的標籤
        if en_index:  
            print(en_index.text.strip())
        # 如果tr內有符合 公司資訊 的標籤
        if th_row and td:
            print(th_row.text.strip(),'-',td.text.strip())
            # 先把公司資訊加到list
            name_list.append(th_row.text.strip())  # 標題
            content_list.append(td.text.strip())   # 內容
            # 轉成{標題:內容}的dict
            company_dict=dict(zip(name_list,content_list))
    #print(company_dict)
    # 把公司名稱當keys 建立更大的dict以做區隔 {公司:{標題:內容}}
    company_dict={name:company_dict}
    # 再加到list
    big_company_list.append(company_dict)
print(big_company_list)

driver.close()

台灣積體電路製造股份有限公司
Taiwan Semiconductor Manufacturing Co., Ltd.

公司類型 - 上市公司
股票代號 - 臺證所：2330（1994年9月5日上市）NYSE：TSM（1997年10月20日上市）LSE：0LCVFWB：TSFA
ISIN - US8740391003
統一編號 - 22099131  （查）
成立 - 1987年2月21日 (1987-02-21)
創辦人 - 張忠謀
代表人物 - 董事長：劉德音副董事長兼總裁：魏哲家
總部 - 臺灣 300096新竹市東區力行六路8號
產業 - 電子工業
產品 - 晶圓代工服務
營業額 - ▲新臺幣1.58兆元（2021年）[1]
稅前盈餘 - ▲新臺幣5,847億元（2020年）
稅後盈餘 - ▲新臺幣5,181億元（2020年）
總資產 - ▲新臺幣2.76兆元（2020年）
資產淨值 - 65,376,200,000 美元 (2020年)
員工人數 - ▲50,000人（2020年）
實收資本額 - 新臺幣259,303,804,580元
市值 - ▲新臺幣16.38兆元（2021年2月5日）
每股盈利 - ▲新臺幣19.97元（2020年）
每股資產淨值 - ▲新臺幣71.33元（2020年）
主要股東 - 美國花旗銀行託管台積電存託憑證專戶（21.8%）行政院國家發展基金管理會（6.38%）管理人員群體（0.36%）[2]
主要子公司 - 30家[3]
網站 - www.tsmc.com
叡揚資訊股份有限公司
Galaxy Software Services Corporation
統一編號 - 22425662  （查）
成立 - 1987年
創辦人 - 張培鏞
總部 - 中華民國（臺灣） 台北市中山區德惠街9號5樓
標語口號 - 品質與價值，承諾必實現Quality & value , we are committed
產業 - 資訊服務
員工人數 - 約700人
網站 - gss.com.tw
華碩電腦股份有限公司
ASUSTeK Computer Inc.

華碩位於台北關渡的總部
商業名稱 - 華碩電腦、華碩、ASUS
公司類型 - 上市公司
股票代號 - 臺證所：2357LSE：ASKD
ISIN - TW0002357001
統一編號 -

In [2]:
# 從多層dict一次印出多個df
import pandas as pd

ff=[{'台灣積體電路製造股份有限公司': {'公司類型': '上市公司', '股票代號': '臺證所：2330（1994年9月5日上市）NYSE：TSM（1997年10月20日上市）LSE：0LCVFWB：TSFA', 'ISIN': 'US8740391003', '統一編號': '22099131  （查）', '成立': '1987年2月21日\xa0(1987-02-21)', '創辦人': '張忠謀', '代表人物': '董事長：劉德音副董事長兼總裁：魏哲家', '總部': '臺灣 300096新竹市東區力行六路8號', '產業': '電子工業', '產品': '晶圓代工服務', '營業額': '▲新臺幣1.58兆元（2021年）[1]', '稅前盈餘': '▲新臺幣5,847億元（2020年）', '稅後盈餘': '▲新臺幣5,181億元（2020年）', '總資產': '▲新臺幣2.76兆元（2020年）', '資產淨值': '65,376,200,000 美元 (2020年)', '員工人數': '▲50,000人（2020年）', '實收資本額': '新臺幣259,303,804,580元', '市值': '▲新臺幣16.38兆元（2021年2月5日）', '每股盈利': '▲新臺幣19.97元（2020年）', '每股資產淨值': '▲新臺幣71.33元（2020年）', '主要股東': '美國花旗銀行託管台積電存託憑證專戶（21.8%）行政院國家發展基金管理會（6.38%）管理人員群體（0.36%）[2]', '主要子公司': '30家[3]', '網站': 'www.tsmc.com'}}, {'叡揚資訊股份有限公司': {'統一編號': '22425662  （查）', '成立': '1987年', '創辦人': '張培鏞', '總部': '中華民國（臺灣） 台北市中山區德惠街9號5樓', '標語口號': '品質與價值，承諾必實現Quality & value , we are committed', '產業': '資訊服務', '員工人數': '約700人', '網站': 'gss.com.tw'}}, {'華碩電腦股份有限公司': {'商業名稱': '華碩電腦、華碩、ASUS', '公司類型': '上市公司', '股票代號': '臺證所：2357LSE：ASKD', 'ISIN': 'TW0002357001', '統一編號': '23638777  （查）', '公司前身': '弘碩電腦', '成立': '1989年4月2日', '創辦人': '施崇棠徐世昌[1]廖敏雄謝偉琦童子賢', '代表人物': '董事長：施崇棠副董事長：徐世昌總裁：曾鏘聲共同執行長：許先越、胡書賓', '總部': '中華民國臺灣 112019臺北市北投區立德路15號1樓[2]', '標語口號': '華碩品質，堅若磐石（Rock sold, heart touching）[3]精彩創新，完美品質（INSPIRING INNOVATION, PERSISTENT PERFECTION）追尋無與倫比（In search of incredible）', '業務範圍': '全世界', '產業': '電子電腦硬體物聯網裝置', '產品': '主機板顯示卡音效卡 筆記型電腦桌上型電腦平板電腦 智慧型手機穿戴式裝置顯示器光碟機 網通產品電競產品', '服務': '華碩皇家俱樂部維修中心據點服務一年延長保固專案', '營業額': '▲新臺幣4,128億元（2020年）[4]', '稅前盈餘': '▲新臺幣331.2億元（2020年）', '稅後盈餘': '▲新臺幣265.6億元（2020年）', '總資產': '新臺幣3,965億元（2020年）', '資產淨值': '新臺幣2,146億元（2020年）', '員工人數': '台灣約6,300人，其他約7,500人（2019年）', '實收資本額': '新臺幣7,427,602,800元', '網站': 'www.asus.com'}}]
print(len(ff))

# 建立空list
bb=[list() for n in range(len(ff))]
df=[list() for n in range(len(ff))]

# dd1=ff[0].values()
# dd2=ff[1].values()
# dd1=list(dd1)
# print(dd1[0])
# print(dd1[0].keys())
# df1=pd.DataFrame([dd1[0].values()],columns=dd1[0].keys(),index=ff[0].keys())
# print(df1)

for i,obj in enumerate(ff):
    #print(obj)
    # dict.kes()或values()一定要轉list才能使用
    bb[i]=list(obj.values())
#     print(bb[i][0].keys())
#     print(bb[i][0].values())

    # 橫向印出
    #df=pd.DataFrame([bb[i][0].values()],columns=bb[i][0].keys(),index=obj.keys())
    df[i]=pd.DataFrame(bb[i][0].values(),index=bb[i][0].keys(),columns=obj.keys())
    #print(df[i])

#     print(df[i])
#     print()

df_=df[0]
n=0
for i in range(len(df)):
    #print(df[i])
    print(n)
    n+=1          
    if n<len(df):
        df_=df_.merge(df[n],how='outer',left_index=True,right_index=True) 
        print('-'*30)        
    else:
        break
df_

3
0
------------------------------
1
------------------------------
2


,台灣積體電路製造股份有限公司,叡揚資訊股份有限公司,華碩電腦股份有限公司
ISIN,US8740391003,NaN,TW0002357001
主要子公司,30家[3],NaN,NaN
主要股東,美國花旗銀行託管台積電存託憑證專戶（21.8%）行政院國家發展基金管理會（6.38%）管理人...,NaN,NaN
代表人物,董事長：劉德音副董事長兼總裁：魏哲家,NaN,董事長：施崇棠副董事長：徐世昌總裁：曾鏘聲共同執行長：許先越、胡書賓
公司前身,NaN,NaN,弘碩電腦
公司類型,上市公司,NaN,上市公司
創辦人,張忠謀,張培鏞,施崇棠徐世昌[1]廖敏雄謝偉琦童子賢
員工人數,"▲50,000人（2020年）",約700人,"台灣約6,300人，其他約7,500人（2019年）"
商業名稱,NaN,NaN,華碩電腦、華碩、ASUS
實收資本額,"新臺幣259,303,804,580元",NaN,"新臺幣7,427,602,800元"


In [ ]:

import pandas as pd

ff=[{'台灣積體電路製造股份有限公司': {'公司類型': '上市公司', '股票代號': '臺證所：2330（1994年9月5日上市）NYSE：TSM（1997年10月20日上市）LSE：0LCVFWB：TSFA', 'ISIN': 'US8740391003', '統一編號': '22099131  （查）', '成立': '1987年2月21日\xa0(1987-02-21)', '創辦人': '張忠謀', '代表人物': '董事長：劉德音副董事長兼總裁：魏哲家', '總部': '臺灣 300096新竹市東區力行六路8號', '產業': '電子工業', '產品': '晶圓代工服務', '營業額': '▲新臺幣1.58兆元（2021年）[1]', '稅前盈餘': '▲新臺幣5,847億元（2020年）', '稅後盈餘': '▲新臺幣5,181億元（2020年）', '總資產': '▲新臺幣2.76兆元（2020年）', '資產淨值': '65,376,200,000 美元 (2020年)', '員工人數': '▲50,000人（2020年）', '實收資本額': '新臺幣259,303,804,580元', '市值': '▲新臺幣16.38兆元（2021年2月5日）', '每股盈利': '▲新臺幣19.97元（2020年）', '每股資產淨值': '▲新臺幣71.33元（2020年）', '主要股東': '美國花旗銀行託管台積電存託憑證專戶（21.8%）行政院國家發展基金管理會（6.38%）管理人員群體（0.36%）[2]', '主要子公司': '30家[3]', '網站': 'www.tsmc.com'}}, {'叡揚資訊股份有限公司': {'統一編號': '22425662  （查）', '成立': '1987年', '創辦人': '張培鏞', '總部': '中華民國（臺灣） 台北市中山區德惠街9號5樓', '標語口號': '品質與價值，承諾必實現Quality & value , we are committed', '產業': '資訊服務', '員工人數': '約700人', '網站': 'gss.com.tw'}}]
#print(ff)
#ff=[{'叡揚資訊股份有限公司': {'統一編號': '22425662  （查）', '成立': '1987年', '創辦人': '張培鏞', '總部': '中華民國（臺灣） 台北市中山區德惠街9號5樓', '標語口號': '品質與價值，承諾必實現Quality & value , we are committed', '產業': '資訊服務', '員工人數': '約700人', '網站': 'gss.com.tw'}}]
print(len(ff))

bb=[list() for n in range(len(ff))]

# dd1=ff[0].values()
# dd2=ff[1].values()
# dd1=list(dd1)
# print(dd1[0])
# print(dd1[0].keys())
# df1=pd.DataFrame([dd1[0].values()],columns=dd1[0].keys(),index=ff[0].keys())
# print(df1)

for i,obj in enumerate(ff):
    #print(obj)
    bb[i]=list(obj.values())
    dict1=list(bb[i][0].keys())
    dict2=list(bb[i][0].keys())
    print(bb[i][0].values())
    print(bb[i][0].keys())
    print(bb[i][0].values())
# #     for d in dd:
# #         print(d.keys())

#     df=pd.DataFrame([bb[i][0].values()],columns=bb[i][0].keys(),index=obj.keys())
#     print(df)

# dict3 = { k: [ dict1[k], dict2[k] ] for k in dict1 }

In [ ]:
# merge只會合併相同儲存格
import pandas as pd

ironman6 = pd.DataFrame({'player':['彭政閔','林智勝','郭阜林','詹子賢'],
                        'number':[23,32,5,39]})
print(ironman6)

ironman3 = pd.DataFrame({'name':['彭政閔','郭阜林','林智勝','陳令其'],
                        'team':['兄弟象','統一獅','兄弟象','兄弟象']})
print(ironman3)

pd.merge(ironman6, ironman3, left_on="player", right_on="name").drop('name',axis=1)

In [ ]:
number = pd.Series({'taipei':200, 'taichung': 300, 'changhua': 400, 'kaohsiung' : 150})
mayor = pd.Series({'taipei': 'Kui', 'taichung': 'Ha', 'changhua': 'Chin', 'ii' : 'Lui'})
ironman_df = pd.DataFrame({'number':number, 'mayor':mayor})
ironman_df